In [33]:
import bokeh.plotting as bk
from bokeh.layouts import Row, WidgetBox, Column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Paragraph, TextInput, Select, RadioGroup, Panel, Tabs, DataTable, TableColumn
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

import numpy as np
import scipy.signal as signal

bk.output_notebook()

Loading BokehJS ...

In [2]:
class FilterDesignTool():
    
    def __init__(self,
                 width=800,
                 height=600):
        
        self.source = ColumnDataSource({
            'x0': [],
            'y0': [],
            'x1': [],
            'y1': [],
            'x2': [],
            'y2': [],
            'x3': [],
            'y3': [],
            'x4': [],
            'y4': [],
            'x5': [],
            'y5': [],
            'x6': [],
            'y6': []
        })
        
        self.filt_source = ColumnDataSource({'nyquist': [],
                                             'wp': [],
                                             'ws': [],
                                             'gstop': [],
                                             'gpass': [],
                                             'b': [],
                                             'a': []
                                            })
        self.filt_image_source = ColumnDataSource({
            'w': [],
            'h': []
        })
        
        self.filter_figure = bk.figure(width=width, height=height)
        self._init_controls()
        self._update_parameters()
        self._init_plot()
        self._update_filter()
        self._plot_filter()
        self._layout_app()
        
    def _layout_app(self):
        filter_information_widgets = WidgetBox(Paragraph(text="Filter Information"))
        response_type_widgets = WidgetBox(Paragraph(text="Response Type"),
                                          self.response_type)
        design_method_widgets = WidgetBox(Paragraph(text="Design Method"),
                                          self.design_method, self.design_method_select)
        filter_order_widgets = WidgetBox(Paragraph(text="Filter Order"),
                                         self.specify_order,
                                         self.specify_order_input)
        options_widgets = WidgetBox(Paragraph(text="Options"),
                                    self.density_factor)
        frequency_specifications_widgets = WidgetBox(Paragraph(text="Frequency Specifications"),
                                                     self.freq_units, self.fs, self.fpass, self.fstop)
        magnitude_specifications_widgets = WidgetBox(Paragraph(text="Magnitude Specifications"),
                                                     self.mag_units, self.amp_pass, self.amp_stop)

        upper_row = Row(filter_information_widgets,
                        self.filter_figure)
        lower_row = Row(Column(response_type_widgets, design_method_widgets),
                        Column(filter_order_widgets, options_widgets),
                        frequency_specifications_widgets,
                        magnitude_specifications_widgets)

        self.layout = Column(upper_row, lower_row)
        
    def _init_controls(self):
        
        self.response_type = RadioGroup(labels=["Lowpass",
                                        "Highpass",
                                        "Bandpass",
                                        "BandStop",
                                        "Differentiator"],
                                        active=0)
        self.response_type.on_change('active', self.update)
        
        self.design_method = RadioGroup(labels=["IIR", "FIR"], active=0)
        self.design_method_select = Select(options=["Butterworth"],
                                           value="Butterworth",
                                           title="Method:")
        
        self.specify_order = RadioGroup(labels=["Min Order", "Specify Order"], active=0)
        self.specify_order_input = TextInput(value='10')
        
        self.density_factor = TextInput(value="20", title="Density Factor")
        
        self.freq_units = Select(value='Hz', title='Units:', options=['Hz', 'Normalised (0-1)'])
        self.fs = TextInput(value='1000.0', title='Fs:')
        self.fs.on_change('value', self.update)
        self.fpass = TextInput(value='100.0', title='Fpass:')
        self.fstop = TextInput(value='250.0', title='Fstop:')
        
        self.mag_units = Select(value='dB', title='Magnitude Units:', options=['dB'])
        self.amp_pass = TextInput(value='1.0', title='Apass:')
        self.amp_stop = TextInput(value='80.0', title='Astop:')
        
        controls = [self.design_method_select,
                    self.specify_order_input,
                    self.density_factor,
                    self.freq_units,
                    self.fs,
                    self.fpass,
                    self.fstop,
                    self.mag_units,
                    self.amp_pass,
                    self.amp_stop
                    ]
        radio_controls = [self.response_type,
                          self.design_method,
                          self.specify_order]
        
        [n.on_change('value', self.update) for n in controls]
        [n.on_change('active', self.update) for n in radio_controls]
        
    def _update_parameters(self):
        
        x = {
            'x0': [float(self.fs.value)/2, float(self.fs.value)/2],
            'y0': [float(self.amp_stop.value)* -1.0, 0],
            'x1': [float(self.fstop.value), float(self.fstop.value)],
            'y1': [float(self.amp_stop.value)* -1.0, float(self.amp_pass.value) * -0.5],
            'x2': [float(self.fpass.value), float(self.fpass.value)],
            'y2': [float(self.amp_stop.value)* -1.0, float(self.amp_pass.value) * -0.5],
            'x3': [0, 0],
            'y3': [float(self.amp_stop.value)*-1.0, float(self.amp_pass.value) * -0.5],
            'x4': [0, float(self.fpass.value)],
            'y4': [float(self.amp_pass.value) * -1.0, float(self.amp_pass.value) * -1.0],
            'x5': [float(self.fstop.value), float(self.fs.value)/2],
            'y5': [float(self.amp_stop.value)* -1.0, float(self.amp_stop.value) * -1.0],
            'x6': [0, float(self.fpass.value)],
            'y6': [float(self.amp_pass.value) * 0.0, float(self.amp_pass.value) * 0.0]
        }
        
        self.source.data = x
        
    def _init_plot(self):
        
        self.filter_figure.line('x0', 'y0', source=self.source)
        self.filter_figure.line('x1', 'y1', source=self.source)
        self.filter_figure.line('x2', 'y2', source=self.source)
        self.filter_figure.line('x1', 'y3', source=self.source)
        self.filter_figure.line('x4', 'y4', source=self.source)
        self.filter_figure.line('x5', 'y5', source=self.source)
        self.filter_figure.line('x6', 'y6', source=self.source)
        
        self.filter_figure.xaxis.axis_label = 'Frequency {0}'.format(self.freq_units.value)
        self.filter_figure.yaxis.axis_label = 'Amplitude {0}'.format(self.mag_units.value)
            
    def _update_filter(self):
        
        if self.freq_units.value == 'Hz':
            x = {'nyquist': [float(self.fs.value) / 2],
                 'wp': [[float(wp) / (0.5*float(self.fs.value)) for wp in self.fpass.value.split(',')]],
                 'ws': [[float(ws) / (0.5*float(self.fs.value)) for ws in self.fstop.value.split(',')]],
                 'gstop': [float(self.amp_stop.value)],
                 'gpass': [float(self.amp_pass.value)],
                 'b': [],
                 'a': []}
            b, a = signal.iirdesign(wp=x['wp'],
                                      ws=x['ws'],
                                      gstop=x['gstop'][0],
                                      gpass=x['gpass'][0],
                                      ftype='ellip')
            x['b'] = [b]
            x['a'] = [a]
            w, h = signal.freqz(b, a)
            w_norm = x['nyquist'][0] * w / max(w)
            h_db = 20 * np.log10(abs(h))
            
            self.filt_image_source.data = {'w': w_norm,
                                           'h': h_db}
            
            self.filt_source.data = x
        
    def _plot_filter(self):
        self.filter_figure.line('w',
                                'h',
                                source=self.filt_image_source,
                                line_color='black')
        
    def update(self, old, new, attr):
        self._update_parameters()
        self._update_filter()
    
    def modify_doc(self, doc):
        doc.add_root(self.layout)
    

In [3]:
p = FilterDesignTool()
handler = FunctionHandler(p.modify_doc)
app = Application(handler)

bk.show(app)

In [52]:
class FilterDesignTool():
    
    iir_filters = {
        'Butterworth': 'butter',
        'Chebyshev I': 'cheby1',
        'Chebyshev II': 'cheby2',
        'Cauer/elliptic': 'ellip',
        'Bessel/Thomson': 'bessel'
    }
    fir_windows = {
        'Boxcar': 'boxcar',
        'Triangle': 'triang',
        'Blackman': 'blackman',
        'Hamming': 'hamming',
        'Hanning': 'hann',
        'Bartlett': 'bartlett',
        'Flattop': 'flattop',
        'Parzen': 'parzen',
        'Bohman': 'bohman',
        'Blackman/harris': 'blackmanharris',
        'Nuttal': 'nuttall',
        'Bart/Hann': 'barthann',
        'Kaiser': 'kaiser'
    }
    
    source = ColumnDataSource({
        'ftype': [],
        'subtype': [],
        'fs': [],
        'nyquist': [],
        'wp': [],
        'ws': [],
        'gpass': [],
        'gstop': [],
        'b': [],
        'a': [],
        'order': [],
        'w': [],
        'h': []
    })

    def __init__(self,
                 width=800,
                 height=600):
                
        self.amp_response_fig = bk.figure(width=width, height=int(height/4))
        self.phase_response_fig = bk.figure(width=width, height=int(height/4))
        self.impulse_response_fig = bk.figure(width=width, height=int(height/4))
        self.step_response_fig = bk.figure(width=width, height=int(height/4))
        
        self._init_controls()
               
        columns = [
            TableColumn(field=key, title=key) for key in self.source.data.keys()
        ]
        
        
        self.filter_table = DataTable(source=self.source, columns=columns)
        
        filter_information_widgets = WidgetBox(Paragraph(text="Filter Information"))
        response_type_widgets = WidgetBox(Paragraph(text="Response Type"),
                                          self.response_type)
        design_method_widgets = WidgetBox(Paragraph(text="Design Method"),
                                          self.design_method, self.design_method_select)
        filter_order_widgets = WidgetBox(Paragraph(text="Filter Order"),
                                         self.specify_order,
                                         self.specify_order_input)
        options_widgets = WidgetBox(Paragraph(text="Options"),
                                    self.density_factor)
        frequency_specifications_widgets = WidgetBox(Paragraph(text="Frequency Specifications"),
                                                     self.freq_units, self.fs, self.fpass, self.fstop)
        magnitude_specifications_widgets = WidgetBox(Paragraph(text="Magnitude Specifications"),
                                                     self.mag_units, self.amp_pass, self.amp_stop)
        
        self.layout = Row(
            Column(self.amp_response_fig,
                   self.phase_response_fig,
                   self.impulse_response_fig,
                   self.step_response_fig,
                   WidgetBox(Paragraph(text="Filter Information")),
                   WidgetBox(self.filter_table)),
            Column(filter_information_widgets,
                   response_type_widgets,
                   design_method_widgets,
                   filter_order_widgets),
            Column(options_widgets,
                   frequency_specifications_widgets,
                   magnitude_specifications_widgets)
        )
        
        #self._init_controls()
        #self._update_parameters()
        #self._init_plot()
        #self._update_filter()
        #self._plot_filter()
        #self._layout_app()
        
    
        
    def _layout_app(self):
            
        filter_information_widgets = WidgetBox(Paragraph(text="Filter Information"))
        response_type_widgets = WidgetBox(Paragraph(text="Response Type"),
                                          self.response_type)
        design_method_widgets = WidgetBox(Paragraph(text="Design Method"),
                                          self.design_method, self.design_method_select)
        filter_order_widgets = WidgetBox(Paragraph(text="Filter Order"),
                                         self.specify_order,
                                         self.specify_order_input)
        options_widgets = WidgetBox(Paragraph(text="Options"),
                                    self.density_factor)
        frequency_specifications_widgets = WidgetBox(Paragraph(text="Frequency Specifications"),
                                                     self.freq_units, self.fs, self.fpass, self.fstop)
        magnitude_specifications_widgets = WidgetBox(Paragraph(text="Magnitude Specifications"),
                                                     self.mag_units, self.amp_pass, self.amp_stop)

        upper_row = Row(filter_information_widgets,
                        self.filter_figure)
        lower_row = Row(Column(response_type_widgets, design_method_widgets),
                        Column(filter_order_widgets, options_widgets),
                        frequency_specifications_widgets,
                        magnitude_specifications_widgets)

        self.layout = Column(upper_row, lower_row)
        
    def _init_controls(self):
        
        self.response_type = RadioGroup(labels=["Lowpass",
                                        "Highpass",
                                        "Bandpass",
                                        "BandStop",
                                        "Differentiator"],
                                        active=0)
        self.response_type.on_change('active', self.update)
        
        self.design_method = RadioGroup(labels=["IIR", "FIR"], active=0)
        self.design_method_select = Select(options=["Butterworth"],
                                           value="Butterworth",
                                           title="Method:")
        
        self.specify_order = RadioGroup(labels=["Min Order", "Specify Order"], active=0)
        self.specify_order_input = TextInput(value='10')
        
        self.density_factor = TextInput(value="20", title="Density Factor")
        
        self.freq_units = Select(value='Hz', title='Units:', options=['Hz', 'Normalised (0-1)'])
        self.fs = TextInput(value='1000.0', title='Fs:')
        self.fs.on_change('value', self.update)
        self.fpass = TextInput(value='100.0', title='Fpass:')
        self.fstop = TextInput(value='250.0', title='Fstop:')
        
        self.mag_units = Select(value='dB', title='Magnitude Units:', options=['dB'])
        self.amp_pass = TextInput(value='1.0', title='Apass:')
        self.amp_stop = TextInput(value='80.0', title='Astop:')
        
        controls = [self.design_method_select,
                    self.specify_order_input,
                    self.density_factor,
                    self.freq_units,
                    self.fs,
                    self.fpass,
                    self.fstop,
                    self.mag_units,
                    self.amp_pass,
                    self.amp_stop
                    ]
        radio_controls = [self.response_type,
                          self.design_method,
                          self.specify_order]
        
        [n.on_change('value', self.update) for n in controls]
        [n.on_change('active', self.update) for n in radio_controls]
        
    def _update_parameters(self):
        pass
        
    def _init_plot(self):
        pass
    
    def _update_filter(self):
        
        if self.freq_units.value == 'Hz':
            x = {'nyquist': [float(self.fs.value) / 2],
                 'wp': [[float(wp) / (0.5*float(self.fs.value)) for wp in self.fpass.value.split(',')]],
                 'ws': [[float(ws) / (0.5*float(self.fs.value)) for ws in self.fstop.value.split(',')]],
                 'gstop': [float(self.amp_stop.value)],
                 'gpass': [float(self.amp_pass.value)],
                 'b': [],
                 'a': []}
            b, a = signal.iirdesign(wp=x['wp'],
                                    ws=x['ws'],
                                    gstop=x['gstop'][0],
                                    gpass=x['gpass'][0],
                                    ftype='ellip')
            x['b'] = [b]
            x['a'] = [a]
            w, h = signal.freqz(b, a)
            w_norm = x['nyquist'][0] * w / max(w)
            h_db = 20 * np.log10(abs(h))
            
            self.filt_image_source.data = {'w': w_norm,
                                           'h': h_db}
            
            self.filt_source.data = x
        
    def _plot_filter(self):
        self.filter_figure.line('w',
                                'h',
                                source=self.filt_image_source,
                                line_color='black')
        
    def update(self, old, new, attr):
        self._update_parameters()
        self._update_filter()
    
    def modify_doc(self, doc):
        doc.add_root(self.layout)
    

In [53]:
p = FilterDesignTool()
handler = FunctionHandler(p.modify_doc)
app = Application(handler)

bk.show(app)

In [51]:
DataTable?

Init signature: DataTable(**kw)
Docstring:     
Two dimensional grid for visualisation and editing large amounts
of data.
Init docstring:         
File:           ~/anaconda3/envs/filter-design-tool/lib/python3.7/site-packages/bokeh/models/widgets/tables.py
Type:           MetaModel


In [8]:
signal.firwin?

Signature: signal.firwin(numtaps, cutoff, width=None, window='hamming', pass_zero=True, scale=True, nyq=None, fs=None)
Docstring:
FIR filter design using the window method.

This function computes the coefficients of a finite impulse response
filter.  The filter will have linear phase; it will be Type I if
`numtaps` is odd and Type II if `numtaps` is even.

Type II filters always have zero response at the Nyquist frequency, so a
ValueError exception is raised if firwin is called with `numtaps` even and
having a passband whose right end is at the Nyquist frequency.

Parameters
----------
numtaps : int
    Length of the filter (number of coefficients, i.e. the filter
    order + 1).  `numtaps` must be even if a passband includes the
    Nyquist frequency.
cutoff : float or 1D array_like
    Cutoff frequency of filter (expressed in the same units as `nyq`)
    OR an array of cutoff frequencies (that is, band edges). In the
    latter case, the frequencies in `cutoff` should be positive an

In [9]:
signal.get_window?

Signature: signal.get_window(window, Nx, fftbins=True)
Docstring:
Return a window.

Parameters
----------
window : string, float, or tuple
    The type of window to create. See below for more details.
Nx : int
    The number of samples in the window.
fftbins : bool, optional
    If True (default), create a "periodic" window, ready to use with
    `ifftshift` and be multiplied by the result of an FFT (see also
    `fftpack.fftfreq`).
    If False, create a "symmetric" window, for use in filter design.

Returns
-------
get_window : ndarray
    Returns a window of length `Nx` and type `window`

Notes
-----
Window types:

    `boxcar`, `triang`, `blackman`, `hamming`, `hann`, `bartlett`,
    `flattop`, `parzen`, `bohman`, `blackmanharris`, `nuttall`,
    `barthann`, `kaiser` (needs beta), `gaussian` (needs standard
    deviation), `general_gaussian` (needs power, width), `slepian`
    (needs width), `dpss` (needs normalized half-bandwidth),
    `chebwin` (needs attenuation), `exponential` 